<a href="https://colab.research.google.com/github/yukiharada1228/neural_network/blob/main/BERT%E3%81%AE%E5%88%A9%E7%94%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install transformers fugashi ipadic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 613.3/613.3 KB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 19.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 5.5 MB/s eta 0:00:00


In [2]:
from transformers import AutoModel, AutoTokenizer
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity

tokenizer = AutoTokenizer.from_pretrained("cl-tohoku/bert-base-japanese")
model = AutoModel.from_pretrained("cl-tohoku/bert-base-japanese")

# 変換する文章をリストで定義
texts = [
    "イケメンがいなくて困ってます．ぴえん．",
    "丸善の商品は定価よりも高かったりしますか？",
    "授業の退出時にカードリーダーに学生証をタッチし忘れた場合、欠席扱いになりますか？また、授業の担当の先生に連絡した方がいいですか？",
    "中部大学の奨学金って重複して取れるんですか？勉強の三十万円と部活とかの十万円",
    "中部大学にはパソコンのマウスやUSBとかを買える所ありますか？",
    "こんなに荒れるんですね．民度がわかります．",
    "夜遅くまで勉強したいのですが基本的に何時まで大学に居ても大丈夫でしょうか",
    "相談会などにいってないのですがどうやったらサークルに入れますか",
    "コモンズの使用時間は９時からですが、9時前に入って勉強している人もいます。良いのでしょうか？9時前でもコモンズ3階のコピー機を使ったりし良いんですか",
    "お昼めっちゃ混んでてビビってます。なにたべましたか？",
    "担当してる教授がどんな研究をしているか知りたいんですがどうやって調べられますか？",
    "眠くて困ってます．",
    "ロッカーを使用している方のほとんどがダイアル式なんですがいいんでしょうか？",
    "履修追加したときの連絡って、直接会って追加したのでよろしくお願いしますって言いに行くのは迷惑ですか？",
    "スマホの充電ができる場所ってあったりしますか？",
    "スマホで中部大学のワイファイにつなぐ方法を教えて！",
    "必修じゃない専門科目って取らないとやばいですかね？",
    "昨日一限で朝行ったときバス混んでたんですが　あれ毎日なんですかね",
    "中部大学にお風呂とかってあるんですか？",
]

# 特徴ベクトルを取得
features_list = []
for text in tqdm(texts):
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model(**inputs)
    features = outputs.last_hidden_state.mean(dim=1).squeeze().detach().numpy()
    # ノルムを1に正規化
    features_norm = np.linalg.norm(features)
    if features_norm > 0:
        features = features / features_norm
    features_list.append(features)

# 特徴ベクトルをnumpy配列に変換
features_array = np.array(features_list)

# 重心を計算
center = features_array.mean(axis=0)

# 特徴量と中心点のコサイン類似度を計算
similarities = cosine_similarity(features_array, center.reshape(1, -1)).flatten()

# 類似度が高い順にデータをソート
sorted_indices = similarities.argsort()[::-1]

for i, text in enumerate(np.array(texts)[sorted_indices]):
    print(f'No.{i}: {text}')

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 19/19 [00:07<00:00,  2.68it/s]

No.0: コモンズの使用時間は９時からですが、9時前に入って勉強している人もいます。良いのでしょうか？9時前でもコモンズ3階のコピー機を使ったりし良いんですか
No.1: ロッカーを使用している方のほとんどがダイアル式なんですがいいんでしょうか？
No.2: 中部大学にお風呂とかってあるんですか？
No.3: 相談会などにいってないのですがどうやったらサークルに入れますか
No.4: 担当してる教授がどんな研究をしているか知りたいんですがどうやって調べられますか？
No.5: 夜遅くまで勉強したいのですが基本的に何時まで大学に居ても大丈夫でしょうか
No.6: 必修じゃない専門科目って取らないとやばいですかね？
No.7: スマホの充電ができる場所ってあったりしますか？
No.8: 授業の退出時にカードリーダーに学生証をタッチし忘れた場合、欠席扱いになりますか？また、授業の担当の先生に連絡した方がいいですか？
No.9: 中部大学の奨学金って重複して取れるんですか？勉強の三十万円と部活とかの十万円
No.10: 昨日一限で朝行ったときバス混んでたんですが　あれ毎日なんですかね
No.11: 中部大学にはパソコンのマウスやUSBとかを買える所ありますか？
No.12: お昼めっちゃ混んでてビビってます。なにたべましたか？
No.13: 丸善の商品は定価よりも高かったりしますか？
No.14: こんなに荒れるんですね．民度がわかります．
No.15: 履修追加したときの連絡って、直接会って追加したのでよろしくお願いしますって言いに行くのは迷惑ですか？
No.16: イケメンがいなくて困ってます．ぴえん．
No.17: スマホで中部大学のワイファイにつなぐ方法を教えて！
No.18: 眠くて困ってます．
